In [172]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras import models
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,Normalizer, MinMaxScaler
from sklearn.cross_validation import cross_val_score
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

In [173]:
df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\DC_ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name
day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)

y=df['Sales']

In [174]:
df.head()
df.shape

(942, 16)

In [175]:
X=df.drop(['id','DayOfWeek','Customers','Sales'],inplace=False,axis=1)
X.head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,10,2013,1,0,0,0,0,0,1,0
1,10,0,10,2013,1,0,0,0,0,0,0,1
2,10,0,8,2013,1,0,0,0,0,1,0,0
3,10,0,8,2013,1,1,0,0,0,0,0,0
4,10,0,1,2013,1,0,0,1,0,0,0,0


In [176]:
for obs in range(1,8):
    X["Sales_T"+str(obs)]=df['Sales'].shift(obs)


In [177]:
X['Mov_avg']=pd.rolling_mean(df['Sales'], window=7).shift(1) 
y=y[7:]
X=X[7:]
y.reset_index(drop=True, inplace=True)
X.reset_index(drop=True, inplace=True)
X.head()



,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7,Mov_avg
0,10,10,1,2013,1,0,0,0,0,0,1,0,89391.0,0.0,43727.0,55606.0,52848.0,60436.0,0.0,43144.000000
1,10,10,1,2013,1,0,0,0,0,0,0,1,72552.0,89391.0,0.0,43727.0,55606.0,52848.0,60436.0,53508.571429
2,10,10,1,2013,1,0,0,0,0,1,0,0,64297.0,72552.0,89391.0,0.0,43727.0,55606.0,52848.0,54060.142857
3,10,10,1,2013,1,1,0,0,0,0,0,0,64112.0,64297.0,72552.0,89391.0,0.0,43727.0,55606.0,55669.285714
4,10,0,0,2013,1,0,0,1,0,0,0,0,65006.0,64112.0,64297.0,72552.0,89391.0,0.0,43727.0,57012.142857


In [178]:
std= preprocessing.StandardScaler().fit(X)
X = std.transform(X)
X

array([[ 0.45458921,  1.26667155, -0.27211318, ...,  0.30299245,
        -1.93174939, -1.17922332],
       [ 0.45458921,  1.26667155, -0.27211318, ...,  0.02171119,
         0.30417536,  0.16166526],
       [ 0.45458921,  1.26667155, -0.27211318, ...,  0.1239481 ,
         0.02344538,  0.23302334],
       ..., 
       [ 0.45458921,  1.26667155,  2.70169512, ...,  0.16046129,
         0.06451157,  0.10254212],
       [ 0.45458921,  1.26667155,  2.70169512, ...,  0.07616587,
         0.16192354,  0.45234637],
       [ 0.45458921,  1.26667155,  2.70169512, ..., -0.27280379,
         0.07779334,  0.76280296]])

In [179]:
cv=10

RF=RandomForestRegressor();
rf_models=[];
rf_models=(cross_val_score(RF,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0) )


GB=ensemble.GradientBoostingRegressor(n_estimators= 500,max_depth= 40, min_samples_split=20,
          learning_rate= 0.01, loss= 'ls');
gb_models=[];
gb_models=(cross_val_score(GB,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
   

XG=xgb.XGBRegressor( #learning_rate =0.1, n_estimators=1000, max_depth=10,
        #min_child_weight=1,gamma=0, subsample=0.8,colsample_bytree=0.8,nthread=4,scale_pos_weight=1,seed=27  
)
xg_models=[]
xg_models=(cross_val_score(XG,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )

In [180]:
def create_MLP():
    mlp=models.Sequential()
    mlp.add(Dense(20, input_dim=X.shape[1], activation='relu'))
    #mlp.add(Dense(20, activation='relu'))
    mlp.add(Dense(30, activation='relu'))
    mlp.add(Dense(10, activation='relu'))
    
    mlp.add(Dense(1, activation='relu'))
    mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
    return mlp

MLP = KerasRegressor(build_fn=create_MLP,
                               epochs=300,
                               batch_size=10,
                               verbose=0)

mlp_models=[]
mlp_models=(cross_val_score(MLP,X,y,cv=cv).mean().round(0)  )

In [181]:
mae=-pd.DataFrame(np.column_stack((mlp_models,rf_models,gb_models,xg_models)) )
mae=mae.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG'})
mae

,MLP,RF,GB,XG
0,3525.0,3210.0,3209.0,3331.0


In [182]:


best=mae.idxmin(axis=1)
best.astype('category')
mae["Best"]=best
mae

,MLP,RF,GB,XG,Best
0,3525.0,3210.0,3209.0,3331.0,GB


In [183]:
def RF_final(X, y):
    tree=RF
    tree.fit(X,y)
    pred=tree.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'RF'})
    return pred

def MLP_final(X,y):
    model = MLP
    model.fit(X,y)
    pred=model.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'MLP'})
    return pred

def GB_final(X,y):
    model = GB
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'GB'})
    return pred

def XG_final(X,y):
    model=XG
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'XG'})
    return pred

In [184]:
#Using aggregated SKU sales data, the best performaning model is selected based on Train/Test. 
#Then best model is used to predict full data set (but still only trained on X_train) 


bm_dict={}
for sku in range(0,1):        
        if  mae["Best"][0]=="RF":
            bm_dict[sku]=RF_final(X,y);
                    
        elif mae["Best"][0]=="NN":
            bm_dict[sku]=MLP_final(X,y);
                                
        elif mae["Best"][0]=="GB":
            bm_dict[sku]=GB_final(X,y); 
                        
        elif mae["Best"][0]=="XG":
            bm_dict[sku]=XG_final(X,y);

#bm_dict1=bm_dict[0][800:]
#bm_dict1.reset_index(drop=True, inplace=True)

In [185]:
# Ceate Phase 2 data set which is comprised of both aggregate SKU data and forecasts (no epirical sales data)
dc_pred=pd.DataFrame(bm_dict[0])
sku_pred=pd.read_csv('C:\\Users\\lengada1\\NCSU\\prediction_skus.csv')
sku_pred=sku_pred.iloc[:,1:]
X2=pd.concat([sku_pred,dc_pred,pd.DataFrame(X)],axis=1)

rng=range(0,33)
new_cols =  ['' + str(i) for i in rng]
X2.columns = new_cols[:32]
std2= preprocessing.StandardScaler().fit(X2)
X2 = std2.transform(X2)

In [186]:
cv=10

rf2_models=[];
rf2_models=(cross_val_score(RF,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0) )

gb2_models=[];
gb2_models=(cross_val_score(GB,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
   

xg2_models=[];
xg2_models=(cross_val_score(XG,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )

mlp2_models=[];
mlp2_models=(cross_val_score(MLP,X,y,cv=cv).mean().round(0)  )

In [187]:
mae2=-pd.DataFrame(np.column_stack((mlp2_models,rf2_models,gb2_models,xg2_models)) )
mae2=mae2.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG'})
mae2

,MLP,RF,GB,XG
0,3438.0,459.0,474.0,445.0


In [188]:
#Using forecasted data and no empirical sales data, the best performaning model is selected based on Train/Test. 

## NOTE this improvement is not the true delta.  Only a final forecast between Stage 1 vs 2 will tell. 


best2=mae2.idxmin(axis=1)
best2.astype('category')
mae2["Best"]=best2
both_mae=pd.concat([mae,mae2],axis=0)
both_mae.reset_index(drop=True, inplace=True)

# Phase 1 & 2 DC Models compared based on Test set MAE scores. 
both_mae['min']=both_mae.min(axis=1)
both_mae['improvement']= ((both_mae['min'][0]-both_mae['min'][1])/both_mae['min'][0] ).round(3)
both_mae

,MLP,RF,GB,XG,Best,min,improvement
0,3525.0,3210.0,3209.0,3331.0,GB,3209.0,0.861
1,3438.0,459.0,474.0,445.0,XG,445.0,0.861


In [189]:
bm2_dict={}
for sku in range(0,1):
        if  mae2["Best"][0]=="RF":
            bm2_dict[sku]=RF_final(X2,y);
                    
        elif mae2["Best"][0]=="NN":
            bm2_dict[sku]=MLP_final(X2,y);
                                
        elif mae2["Best"][0]=="GB":
            bm2_dict[sku]=GB_final(X2,y); 
                        
        elif mae2["Best"][0]=="XG":
            bm2_dict[sku]=XG_final(X2,y);    

In [190]:
Stage2_improvement=mean_absolute_error(y, bm2_dict[0])/mean_absolute_error(y, bm_dict[0])
Stage2_improvement

0.57983887032871684

In [191]:
tier_pred=pd.concat([bm_dict[0],bm2_dict[0],y],axis=1)
tier_pred.columns = ['DC_Phase1', 'DC_Phase2',"Sales"]
tier_pred.to_csv('C:\\Users\\lengada1\\NCSU\\DC_tier_predictions.csv')
tier_pred.round()

,DC_Phase1,DC_Phase2,Sales
0,71820.0,72442.0,72552
1,64198.0,64384.0,64297
2,63664.0,63301.0,64112
3,64958.0,64922.0,65006
4,43276.0,43136.0,43300
5,344.0,42.0,0
6,50911.0,50733.0,50984
7,48068.0,48082.0,48246
8,46164.0,45733.0,45400
9,47221.0,47048.0,46978


In [192]:
# END NOTEBOOK

In [193]:
# The rest of this notebook is just a reference and aid to allow collaborators to save models and retrive and apply them without retraining. 
# The file names are not correct, thus just a reference. 

# also look at joblib.  https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
model_json = model.to_json()
with open("NCSU_DC.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("NCSU_DC.h5")
print("Saved model to disk")

NameError: name 'model' is not defined

In [ ]:
# load json and create model
json_file = open('NCSU_DC.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("NCSU_DC.h5")
print("Loaded model from disk")

In [ ]:
# evaluate loaded model on test data
loaded_model.compile(loss='mse', optimizer='adam', metrics=['mae','mse'])
score = loaded_model.evaluate(Xs, y, verbose=0)
print("%s: %.0f"%  (loaded_model.metrics_names[1], score[1]))
print("%s: %.1f"%  (loaded_model.metrics_names[2], score[2]/10000000))